# Analysis of Transfer Students at the University of Washington

Created by Lovenoor (Lavi) Aulck. Part of project examining persistence and performance of community college transfers. Published in PLOS One (currently under review - link will be added once published).  

## Overview of notebook

This notebook contains code to conduct an analysis of the persistence and performance of transfer students using aggregated student data.

## Data format

The notebook assumes input data in a csv format in a file named 'data.csv' with one entry per student and the following columns:

- system_key: a unique identifier for each student (numeric or string)
- first_yr_tran: the student's first calendar year with a transcript grade (numeric)
- first_qtr_tran: the student's first term with a calendar grade (assumes quarter system numbered 1-4 with 1 being the first quarter in the calendar year). Calculated as first year * 4 + first qtr. (numeric)
- last_schooling: the student's previous institution prior to joining the university. 1 = high school; 2 = community college; 3 = other 4-year institution. (numeric)
- trans_credits: number of credits transferred into the university by the student
- graduated: a binary indicator of whether the student graduated from the university in good standing with an undergraduate degree
- total_time_in_qtrs: the duration the student was enrolled in the university between their first and last quarters (in calendar quarters, not academic quarters). (numeric)
- num_qtrs_tran: the number of quarters the student was enrolled in the university (in academic quarters, not calendar quarters). (numeric)
- birth_yr: student's birth year. (numeric)
- gender: student's gender. (string)
- resident: student's resident status. All values < 3 represent state residents while all values >= 3 represent out-of state students. (numeric)
- ethnic: student's ethnicity. (string)
- hispanic: student's indentification as a hispanic student. (string)
- satv: student's SAT verbal score (assumes a 0 score indicates missing). (numeric)
- satm: student's SAT math score (assumes a 0 score indicates missing). (numeric)
- act: student's ACT score (assumes a 0 score indicates missing). (numeric)

### Import packages

In [1]:
#Only two packages are required to run this analysis, assuming the input data is formatted correctly
import pandas as pd;
import numpy as np;

pd.set_option('display.height', 500) #change display of pandas for notebook
pd.set_option('display.max_rows', 500)

fileSave = False #whether to save files for major shift heatmaps

height has been deprecated.



### Import data

In [2]:
dataAll = pd.read_csv('data.csv', low_memory = False)
dataAll = dataAll.set_index(dataAll['system_key'])

In [3]:
len(dataAll)

69582

### Take data of interest

In [4]:
dataPre2007 = dataAll[dataAll['first_qtr_tran'] > (1997 * 4)] #take only students who started within selected time frame
dataPre2007 = dataPre2007[dataPre2007['first_qtr_tran'] < ((2007 * 4) + 1)]
dataPre2007 = dataPre2007[dataPre2007.last_schooling < 4] #university has categorizations for previous schooling that number > 3
dataPre2007 = dataPre2007[dataPre2007.last_schooling > 0]

In [5]:
len(dataPre2007)

69118

In [6]:
dataAll.first_yr_tran.value_counts() #get counts of students by first year

2000.0    8261
2001.0    8209
2006.0    8081
1999.0    7941
2002.0    7568
2004.0    7461
2003.0    7460
2005.0    7354
1998.0    7247
Name: first_yr_tran, dtype: int64

### Calculate time to grad

In [7]:
#university assumes 15 cr per quarter (with 12 cr per qtr considered full-time enrollment). This 15 credits is derived from
#taking 180 credits (requirement for a bachelor's degree) and dividing by 12 (4 years of enrollment with 3 quarters per year;
#students typically take the summer off). Window for successful graduation in this project is 6 calendar years.

dataPre2007['qtrs_transferred'] = ((dataPre2007.trans_credits / 15).round(0)).fillna(0) #calculate number of quarters transferred
dataPre2007['exp_grad_time'] = 24 - dataPre2007.qtrs_transferred #24 = 6 years * 4 qtrs per year. Exp_grad_time is adjusted time
#to completion after accounting for transferred credits

#check for graduation = student earned degree in less than 24 calendar quarters. trans_grad_on_time = student graduated on time
dataPre2007['trans_grad_on_time'] = (dataPre2007.exp_grad_time >= (dataPre2007.total_time_in_qtrs)) & dataPre2007.graduated

In [8]:
#aggregate data across quarters for transfer shock calculations
#credits for first two and first three quarters
dataPre2007['first_two_qtr_cr'] = dataPre2007['first_qtr_cr'] + dataPre2007['sec_qtr_cr']
dataPre2007['first_thr_qtr_cr'] = dataPre2007['first_two_qtr_cr'] + dataPre2007['trd_qtr_cr']

#gpa for first two and first three quarters
dataPre2007['first_two_qtr_gpa'] = dataPre2007['first_qtr_cr'] * dataPre2007['first_qtr_gpa']
dataPre2007['first_two_qtr_gpa'] = dataPre2007['first_two_qtr_gpa'] + (dataPre2007['sec_qtr_cr'] * dataPre2007['sec_qtr_gpa'])
dataPre2007['first_thr_qtr_gpa'] = dataPre2007['first_two_qtr_gpa'] + (dataPre2007['trd_qtr_cr'] * dataPre2007['trd_qtr_gpa'])

dataPre2007['first_two_qtr_gpa'] = dataPre2007['first_two_qtr_gpa'] / dataPre2007['first_two_qtr_cr']
dataPre2007['first_thr_qtr_gpa'] = dataPre2007['first_thr_qtr_gpa'] / dataPre2007['first_thr_qtr_cr']

### Set up groupings

In [9]:
#change grouping names for more clean printing of results
groups = {1: 'freshmen', 2: '2_yr_trans', 3: '4_yr_trans'}
dataPre2007['group'] = dataPre2007.last_schooling.map(groups)

In [10]:
#two different column orders to be used when printing
colOrder = groups.values()
colOrder = np.append('total', colOrder)

colOrder2 = groups.values()
colOrder2 = np.append('all', colOrder2)

### Group demographics

As a first step, the below code looks at raw counts of students across demographic categories. It then calculates the percent composition of each group by demographic.

In [11]:
groupCounts = dataPre2007.groupby(['group'])['group'].count() # raw counts of each group
groupCounts.loc['total'] = groupCounts.sum()
groupCounts = groupCounts.to_frame().transpose()[colOrder]
groupCounts.index = ['all']

In [12]:
genderCounts = dataPre2007.groupby(['gender', 'group'])['group'].count() #counts of groups by gender
genderCounts = genderCounts.unstack()
genderCounts['total'] = dataPre2007.groupby(['gender'])['group'].count()
genderCounts = genderCounts[colOrder]

In [13]:
genderPcts = genderCounts / np.sum(genderCounts) * 100 #to calculate percent. This process is repeated for each demo category

In [14]:
dataPre2007['resident_status'] = (dataPre2007.resident < 3).map({True: 'resident', False: 'non-resident'}) #map resident values
#to a binary outcome - resident or non-resident

In [15]:
residentCounts = dataPre2007.groupby(['resident_status', 'group'])['group'].count() #counts of groups by resident status
residentCounts = residentCounts.unstack()
residentCounts['total'] = dataPre2007.groupby(['resident_status'])['group'].count()
residentCounts = residentCounts[colOrder]

In [16]:
residentPcts = residentCounts / np.sum(residentCounts) * 100

In [17]:
ethnicCounts = dataPre2007.groupby(['ethnic', 'group'])['group'].count() #counts of groups by ethnicity
ethnicCounts = ethnicCounts.unstack()
ethnicCounts['total'] = dataPre2007.groupby(['ethnic'])['group'].count()
ethnicCounts = ethnicCounts[colOrder]

In [18]:
ethnicPcts = ethnicCounts / np.sum(ethnicCounts) * 100

In [19]:
hispanicCounts = dataPre2007.groupby(['hispanic', 'group'])['group'].count() #counts of groups by hispanic identification
hispanicCounts = hispanicCounts.unstack()
hispanicCounts['total'] = dataPre2007.groupby(['hispanic'])['group'].count()
hispanicCounts = hispanicCounts[colOrder]

In [20]:
hispanicPcts = hispanicCounts / np.sum(hispanicCounts) * 100

In [21]:
allCounts = groupCounts.append(genderCounts.append(ethnicCounts.append(hispanicCounts.append(residentCounts))))
allCounts #merging counts together

group,total,freshmen,2_yr_trans,4_yr_trans
all,69118,35855,17680,15583
F,36671,19074,9315,8282
M,32401,16758,8360,7283
U,46,23,5,18
AFRO-AM,2011,1017,534,460
AMER-IND,938,480,245,213
ASIAN,16037,9600,3067,3370
CAUCASN,39442,20123,10292,9027
HAW/PAC,448,265,85,98
NOT IND,10242,4370,3457,2415


In [22]:
allPcts = genderPcts.append(ethnicPcts.append(hispanicPcts.append(residentPcts)))
allPcts.round(1) #merging percents together; single decimal place

group,total,freshmen,2_yr_trans,4_yr_trans
F,53.1,53.2,52.7,53.1
M,46.9,46.7,47.3,46.7
U,0.1,0.1,0.0,0.1
AFRO-AM,2.9,2.8,3.0,3.0
AMER-IND,1.4,1.3,1.4,1.4
ASIAN,23.2,26.8,17.3,21.6
CAUCASN,57.1,56.1,58.2,57.9
HAW/PAC,0.6,0.7,0.5,0.6
NOT IND,14.8,12.2,19.6,15.5
HISPANIC,4.3,4.1,4.5,4.5


### Look at ages and entrance exams

Next, we look at ages and entrance exam scores of the entering students

In [23]:
dataPre2007['age_at_start'] = dataPre2007.first_yr_tran - dataPre2007.birth_yr #age at the start of time at university

In [24]:
#first look at ages for all students
hold = pd.DataFrame({'mean':dataPre2007.age_at_start.mean(), 'std':dataPre2007.age_at_start.std()}, index = ['all'])
#then add ages of students by groups. Transposition orients dataframe in correct way
agesAll = hold.append(dataPre2007.groupby(['group'])['age_at_start'].agg(['mean', 'std'])).transpose()
agesAll = agesAll[colOrder2]
agesAll.index.name = 'age'
agesAll.round(2)

,all,freshmen,2_yr_trans,4_yr_trans
age,,,,
mean,20.41,18.37,23.98,21.05
std,4.44,0.70,5.75,4.93


In [25]:
#look at numbers of students who reported standardized test scores. 0 scores represent missing data
satScores = dataPre2007[(dataPre2007.satv > 0) & (dataPre2007.satm > 0)][['satv', 'satm', 'group']]
actScores = dataPre2007[dataPre2007.act > 0][['act', 'group']]

hold = pd.DataFrame({'sat':len(satScores), 'act':len(actScores)}, index = ['all']) #groupby to get counts
satCounts = satScores.groupby(['group'])['group'].count()
actCounts = actScores.groupby(['group'])['group'].count()
stdTestCounts = pd.DataFrame({'sat': satCounts, 'act': actCounts}).append(hold)

stdTestCounts = stdTestCounts.transpose()[colOrder2] #reorganize results
del stdTestCounts.columns.name
stdTestCounts.index.name = 'counts'
stdTestCounts

,all,freshmen,2_yr_trans,4_yr_trans
counts,,,,
act,7748,4627,815,2306
sat,28067,16491,3127,8449


In [26]:
#then, look at scores at aggregate
hold = pd.DataFrame({'mean':satScores.satv.mean(), 'std':satScores.satv.std()}, index = ['all'])
satvAll = hold.append(satScores.groupby(['group'])['satv'].agg(['mean', 'std'])).transpose()[colOrder2]
satvAll['test'] = 'satv'

In [27]:
hold = pd.DataFrame({'mean':satScores.satm.mean(), 'std':satScores.satm.std()}, index = ['all'])
satmAll = hold.append(satScores.groupby(['group'])['satm'].agg(['mean', 'std'])).transpose()[colOrder2]
satmAll['test'] = 'satm'

In [28]:
hold = pd.DataFrame({'mean':actScores.act.mean(), 'std':actScores.act.std()}, index = ['all'])
actAll = hold.append(actScores.groupby(['group'])['act'].agg(['mean', 'std'])).transpose()[colOrder2]
actAll['test'] = 'act'

In [29]:
standTests = satvAll.append(satmAll.append(actAll))
standTests.index.name = 'measure'
standTests.set_index('test', append = True, inplace = True)
standTests = standTests.reorder_levels(['test', 'measure'])
standTests.round(2).unstack()

all        freshmen        2_yr_trans        4_yr_trans       
measure    mean    std     mean    std       mean    std       mean    std
test                                                                      
act       25.19   4.21    25.38   4.15      23.45   4.31      25.42   4.15
satm     600.17  88.11   605.82  85.35     557.94  89.29     604.77  88.78
satv     576.11  94.01   577.76  91.74     548.23  99.35     583.21  94.54

### Find grad info

Next, we look at graduation rates among demographic groups.

In [30]:
gradData6yr = dataPre2007[dataPre2007['trans_grad_on_time']] #graduates within 6 years
ncData = dataPre2007[~dataPre2007['trans_grad_on_time']] #non-completions

In [31]:
if fileSave: #saves graduates and noncompletions to file (for use in generating major-shift heatmaps)
    gradData6yr.to_csv('graduates.csv')
    ncData.to_csv('noncompletions.csv')

In [32]:
#the below is very much similar to the above with demographic only using just graduates
groupCountsGrad = gradData6yr.groupby(['group'])['group'].count()
groupCountsGrad.loc['total'] = groupCountsGrad.sum()
groupCountsGrad = groupCountsGrad.to_frame().transpose()[colOrder]
groupCountsGrad.index = ['all']

In [33]:
genderCountsGrad = gradData6yr.groupby(['gender', 'group'])['group'].count()
genderCountsGrad = genderCountsGrad.unstack()
genderCountsGrad['total'] = gradData6yr.groupby(['gender'])['group'].count()
genderCountsGrad = genderCountsGrad[colOrder]

In [34]:
residentCountsGrad = gradData6yr.groupby(['resident_status', 'group'])['group'].count()
residentCountsGrad = residentCountsGrad.unstack()
residentCountsGrad['total'] = gradData6yr.groupby(['resident_status'])['group'].count()
residentCountsGrad = residentCountsGrad[colOrder]

In [35]:
ethnicCountsGrad = gradData6yr.groupby(['ethnic', 'group'])['group'].count()
ethnicCountsGrad = ethnicCountsGrad.unstack()
ethnicCountsGrad['total'] = gradData6yr.groupby(['ethnic'])['group'].count()
ethnicCountsGrad = ethnicCountsGrad[colOrder]

In [36]:
hispanicCountsGrad = gradData6yr.groupby(['hispanic', 'group'])['group'].count()
hispanicCountsGrad = hispanicCountsGrad.unstack()
hispanicCountsGrad['total'] = gradData6yr.groupby(['hispanic'])['group'].count()
hispanicCountsGrad = hispanicCountsGrad[colOrder]

In [37]:
gradCounts = groupCountsGrad.append(genderCountsGrad.append(ethnicCountsGrad.append(hispanicCountsGrad.append(residentCountsGrad))))
gradPcts = gradCounts / allCounts * 100 #combine all counts of graduates and compare to raw counts. Also convert to attrition rates
gradPcts = 100 - gradPcts
gradPcts.round(1) #single decimal place

group,total,freshmen,2_yr_trans,4_yr_trans
all,23.5,20.9,23.5,29.6
F,22.2,20.2,23.1,25.8
M,25.1,21.7,24.1,34.0
U,13.0,8.7,0.0,22.2
AFRO-AM,31.9,29.0,29.4,41.3
AMER-IND,34.3,34.0,29.0,41.3
ASIAN,22.3,17.9,25.6,31.8
CAUCASN,23.2,21.1,23.4,27.7
HAW/PAC,31.7,31.7,29.4,33.7
NOT IND,23.8,22.7,20.7,30.3


In [38]:
#this looks at attrition rates by cohort (year of first entry into the university)
countsByYear = dataPre2007.groupby(['first_yr_tran', 'group'])['group'].count().unstack()
countsByYear['all'] = dataPre2007.groupby(['first_yr_tran'])['group'].count()
ratesByYear = dataPre2007[dataPre2007.trans_grad_on_time].groupby(['first_yr_tran', 'group'])['group'].count().unstack()
ratesByYear['all'] = dataPre2007[dataPre2007.trans_grad_on_time].groupby(['first_yr_tran'])['group'].count()
ratesByYear = 100 - ((ratesByYear / countsByYear) * 100) #convert to attrition rate
ratesByYear = ratesByYear[colOrder2]

ratesByYear['fresh-2yr_diff'] = ratesByYear.freshmen - ratesByYear['2_yr_trans'] #calculate rate differences by group
ratesByYear['fresh-4yr_diff'] = ratesByYear.freshmen - ratesByYear['4_yr_trans']
ratesByYear['2yr-4yr_diff'] = ratesByYear['2_yr_trans'] - ratesByYear['4_yr_trans']
ratesByYear.round(2)

group,all,freshmen,2_yr_trans,4_yr_trans,fresh-2yr_diff,fresh-4yr_diff,2yr-4yr_diff
first_yr_tran,,,,,,,
1998.0,27.56,24.32,27.40,32.82,-3.07,-8.50,-5.43
1999.0,27.75,23.66,27.35,34.98,-3.69,-11.32,-7.63
2000.0,27.11,23.02,27.51,34.20,-4.49,-11.18,-6.70
2001.0,24.65,22.64,24.14,29.86,-1.50,-7.22,-5.72
2002.0,22.98,21.04,21.63,29.36,-0.59,-8.32,-7.73
2003.0,20.31,18.32,21.95,23.73,-3.63,-5.41,-1.78
2004.0,20.78,18.49,21.77,25.88,-3.28,-7.39,-4.11
2005.0,20.29,18.86,19.88,24.96,-1.02,-6.10,-5.08
2006.0,20.20,18.98,19.31,25.07,-0.33,-6.08,-5.75


In [39]:
#look at the average difference between the groups for the first three and last three cohorts
averageDiffFirstThreeYears = ratesByYear.iloc[:3,-3:].mean()
averageDiffFirstThreeYears

group
fresh-2yr_diff    -3.749870
fresh-4yr_diff   -10.336337
2yr-4yr_diff      -6.586467
dtype: float64

In [40]:
averageDiffLastThreeYears = ratesByYear.iloc[-3:,-3:].mean()
averageDiffLastThreeYears

group
fresh-2yr_diff   -1.545134
fresh-4yr_diff   -6.527132
2yr-4yr_diff     -4.981998
dtype: float64

### Look at the number of quarters enrolled

In [41]:
#look at the number of quarters enrolled by graduating status (trans_grad_on_time = whether student is a graduate or not)
numQtrsEnrolled = dataPre2007.groupby(['group', 'trans_grad_on_time'])['num_qtrs_tran'].agg(['mean', 'median', 'std'])
numQtrsEnrolled = numQtrsEnrolled.unstack().transpose()
numQtrsEnrolled.round(2)

group                      2_yr_trans  4_yr_trans  freshmen
       trans_grad_on_time                                  
mean   False                     5.79        9.84      6.80
       True                      8.31       10.87     13.48
median False                     4.00        9.00      5.00
       True                      8.00       11.00     13.00
std    False                     4.87        6.10      5.33
       True                      2.33        3.50      2.21

In [42]:
#looking at the %age of students who enrolled for exactly three quarters and left the university
float(len(ncData[ncData.num_qtrs_tran == 3])) / len(ncData) * 100

16.5929203539823

In [43]:
#get counts of non-completions by group
ncCounts = ncData.groupby(['group'])['group'].count()
ncCounts['total'] = ncCounts.sum()
ncCounts = ncCounts.to_frame().transpose()[colOrder]
ncCounts.index = ['all']

In [44]:
#look at students who attrite in their first year
yearOneDrops = ncData[ncData.num_qtrs_tran < 4]
propYearOneDrops = yearOneDrops.groupby(['group'])['group'].count()
propYearOneDrops['total'] = propYearOneDrops.sum()
propYearOneDrops = propYearOneDrops.to_frame().transpose()[colOrder]
propYearOneDrops.index = ['all']
propYearOneDrops = propYearOneDrops / ncCounts * 100
propYearOneDrops.round(1)

group,total,freshmen,2_yr_trans,4_yr_trans
all,34.4,37.4,45.4,19.7


In [45]:
#look at the same but students who attrite in their first calendar year (above was academic year)
yearOneDrops = ncData[ncData.total_time_in_qtrs < 4]
propYearOneDrops = yearOneDrops.groupby(['group'])['group'].count()
propYearOneDrops['total'] = propYearOneDrops.sum()
propYearOneDrops = propYearOneDrops.to_frame().transpose()[colOrder]
propYearOneDrops.index = ['all']
propYearOneDrops = propYearOneDrops / ncCounts * 100
propYearOneDrops.round(1)

group,total,freshmen,2_yr_trans,4_yr_trans
all,31.9,36.9,40.9,15.7


### Look at student grades

In [46]:
#get 25th and 7th percentiles for all students and each group
hold = dataPre2007['overall_gpa'].quantile([0.25, 0.75]) #get 25th and 75th percentiles
hold.index.names = ['percentile']
percentiles = dataPre2007.groupby(['group'])['overall_gpa'].quantile([0.25, 0.75])
percentiles.index.names = ['group', 'percentile']
percentiles = percentiles.reorder_levels(['percentile', 'group']).unstack()
percentiles['all'] = hold

In [47]:
#then look at all GPAs at aggregate
hold = pd.DataFrame({'mean':dataPre2007['overall_gpa'].mean(), 
                     'std':dataPre2007['overall_gpa'].std(),
                    'median':dataPre2007['overall_gpa'].median()},
                    index = ['all'])
gpasAll = hold.append(dataPre2007.groupby(['group'])['overall_gpa'].agg([np.mean, np.std, np.median])).transpose()[colOrder2]
gpasAll = gpasAll.append(percentiles)
gpasAll.index.name = 'GPA'
gpasAll = gpasAll[colOrder2]
gpasAll.round(2)

,all,freshmen,2_yr_trans,4_yr_trans
GPA,,,,
mean,3.14,3.12,3.10,3.22
median,3.26,3.24,3.21,3.35
std,0.56,0.56,0.58,0.55
0.25,2.89,2.88,2.84,2.97
0.75,3.53,3.50,3.49,3.61


In [48]:
#look at differences by group
hold = dataPre2007.groupby(['trans_grad_on_time'])['overall_gpa'].quantile([0.25, 0.75]) #get 25th and 75th percentiles
hold.index.names = ['trans_grad_on_time', 'percentile']
hold = hold.reorder_levels(['percentile', 'trans_grad_on_time'])
percentiles = dataPre2007.groupby(['group', 'trans_grad_on_time'])['overall_gpa'].quantile([0.25, 0.75])
percentiles.index.names = ['group', 'trans_grad_on_time', 'percentile']
percentiles = percentiles.reorder_levels(['percentile', 'trans_grad_on_time', 'group']).unstack()
percentiles['all'] = hold

In [49]:
#then get mean, std, and median
hold = dataPre2007.groupby(['trans_grad_on_time'])['overall_gpa'].agg([np.mean, np.std, np.median]).stack().swaplevel(i=-2, j=-1)
gpasAll = dataPre2007.groupby(['group', 'trans_grad_on_time'])['overall_gpa'].agg([np.mean, np.std, np.median])
gpasAll = gpasAll.unstack().transpose()
gpasAll['all'] = hold
gpasAll = gpasAll.append(percentiles) #add percentiles
gpasAll = gpasAll[colOrder2] 

gpasAll.round(2)

group                       all  freshmen  2_yr_trans  4_yr_trans
       trans_grad_on_time                                        
mean   False               2.64      2.57        2.59        2.81
       True                3.29      3.27        3.25        3.39
std    False               0.77      0.78        0.81        0.68
       True                0.38      0.37        0.39        0.36
median False               2.74      2.65        2.70        2.89
       True                3.33      3.31        3.29        3.45
0.25   False               2.17      2.06        2.12        2.38
       True                3.05      3.04        3.00        3.18
0.75   False               3.22      3.17        3.20        3.31
       True                3.57      3.54        3.53        3.66

### Transfer shock

Next, look at student differences after their 1st, 2nd, and 3rd terms of enrollment.

In [50]:
pre1stQtr = dataPre2007[dataPre2007.first_qtr_cr > 11] #include only full-time enrollees in first qtr
pre1stQtr = pre1stQtr[pre1stQtr.post_first_qtr_cr > 11] #include only students who enrolled for at least credit total equaling a subsequent full-time term
pre1stQtrCounts = pre1stQtr.groupby(['group'])['group'].count()
pre1stQtrCounts.loc['all'] = len(pre1stQtr)
pre1stQtrCounts = pre1stQtrCounts.transpose()
pre1stQtrHold = pd.DataFrame({'mean':pre1stQtr[['first_qtr_gpa', 'post_first_qtr_gpa']].mean(), 
                              'std':pre1stQtr[['first_qtr_gpa', 'post_first_qtr_gpa']].std()}, 
                             index = ['first_qtr_gpa', 'post_first_qtr_gpa']).stack()

In [51]:
#get data for each group
pre1stQtr = pre1stQtr.groupby(['group'])['first_qtr_gpa', 'post_first_qtr_gpa'].agg(['mean', 'std']).transpose()
pre1stQtr['all'] = pre1stQtrHold
pre1stQtr = pre1stQtr[colOrder2].unstack()

In [52]:
pre2ndQtr = dataPre2007[dataPre2007.first_two_qtr_cr > 23] #repeat above. Students who averaged full-time enrollment
pre2ndQtr = pre2ndQtr[pre2ndQtr.post_sec_qtr_cr > 11] #and had at least a full-term worth of subsequent credits
pre2ndQtrCounts = pre2ndQtr.groupby(['group'])['group'].count()
pre2ndQtrCounts.loc['all'] = len(pre2ndQtr)
pre2ndQtrCounts = pre2ndQtrCounts.transpose()
pre2ndQtrHold = pd.DataFrame({'mean':pre2ndQtr[['first_two_qtr_gpa', 'post_sec_qtr_gpa']].mean(), 
                              'std':pre2ndQtr[['first_two_qtr_gpa', 'post_sec_qtr_gpa']].std()}, 
                             index = ['first_two_qtr_gpa', 'post_sec_qtr_gpa']).stack()

In [53]:
pre2ndQtr = pre2ndQtr.groupby(['group'])['first_two_qtr_gpa', 'post_sec_qtr_gpa'].agg(['mean', 'std']).transpose()
pre2ndQtr['all'] = pre2ndQtrHold
pre2ndQtr = pre2ndQtr[colOrder2].unstack()

In [54]:
pre3rdQtr = dataPre2007[dataPre2007.first_thr_qtr_cr > 35] #same as above only for first three quarters
pre3rdQtr = pre3rdQtr[pre3rdQtr.post_trd_qtr_cr > 11]
pre3rdQtrCounts = pre3rdQtr.groupby(['group'])['group'].count()
pre3rdQtrCounts.loc['all'] = len(pre3rdQtr)
pre3rdQtrCounts = pre3rdQtrCounts.transpose()
pre3rdQtrHold = pd.DataFrame({'mean':pre3rdQtr[['first_thr_qtr_gpa', 'post_trd_qtr_gpa']].mean(), 
                              'std':pre3rdQtr[['first_thr_qtr_gpa', 'post_trd_qtr_gpa']].std()}, 
                             index = ['first_thr_qtr_gpa', 'post_trd_qtr_gpa']).stack()

In [55]:
pre3rdQtr = pre3rdQtr.groupby(['group'])['first_thr_qtr_gpa', 'post_trd_qtr_gpa'].agg(['mean', 'std']).transpose()
pre3rdQtr['all'] = pre3rdQtrHold
pre3rdQtr = pre3rdQtr[colOrder2].unstack()

In [56]:
pre1stQtr.append(pre2ndQtr.append(pre3rdQtr)).round(3)

group                 all        freshmen        2_yr_trans        4_yr_trans  \
                     mean    std     mean    std       mean    std       mean   
first_qtr_gpa       3.206  0.560    3.192  0.558      3.174  0.556      3.275   
post_first_qtr_gpa  3.194  0.525    3.178  0.529      3.157  0.526      3.269   
first_two_qtr_gpa   3.186  0.512    3.169  0.504      3.160  0.511      3.254   
post_sec_qtr_gpa    3.195  0.527    3.176  0.531      3.175  0.518      3.266   
first_thr_qtr_gpa   3.208  0.466    3.196  0.454      3.180  0.465      3.266   
post_trd_qtr_gpa    3.223  0.506    3.207  0.504      3.206  0.502      3.284   

group                      
                      std  
first_qtr_gpa       0.565  
post_first_qtr_gpa  0.507  
first_two_qtr_gpa   0.526  
post_sec_qtr_gpa    0.516  
first_thr_qtr_gpa   0.494  
post_trd_qtr_gpa    0.512

In [57]:
#counts of students
pd.DataFrame({'1st_qtr': pre1stQtrCounts, '2nd_qtr': pre2ndQtrCounts, '3rd_qtr': pre3rdQtrCounts}).transpose()[colOrder2]

group,all,freshmen,2_yr_trans,4_yr_trans
1st_qtr,47748,26300,10752,10696
2nd_qtr,54400,31530,11060,11810
3rd_qtr,51116,29745,10279,11092
